In [1]:
import os
import mmcv

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

## Need to change code 

### code from mmseg/model/segmentors/encoder_decoder.py

### change code line 282 from seg_pred = seg_logit.argmax(dim=1) to seg_pred = seg_logit 

In [2]:
# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/models/upernet_swin_l.py')
root='../input/mmseg/test/'
epoch = 'latest'

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '../output/mmseg/upernet_swin_l_alldata'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model1 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model1, checkpoint_path, map_location='cpu')

model1.CLASSES = dataset.CLASSES
model1 = MMDataParallel(model1.cuda(), device_ids=[0])

2021-11-01 02:48:56,168 - mmseg - INFO - Loaded 819 images
/opt/ml/segmentation/mmsegmentation/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


Use load_from_local loader


In [3]:
# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/models/upernet_swin_l.py')
root='../input/mmseg/test/'
epoch = 'epoch_45'

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '../output/mmseg/upernet_swin_l_alldata'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model2 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model2, checkpoint_path, map_location='cpu')

model2.CLASSES = dataset.CLASSES
model2 = MMDataParallel(model2.cuda(), device_ids=[0])

2021-11-01 02:49:04,008 - mmseg - INFO - Loaded 819 images


Use load_from_local loader


In [4]:
# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/models/upernet_swin_l.py')
root='../input/mmseg/test/'
epoch = 'epoch_40'

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '../output/mmseg/upernet_swin_l_alldata'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model3 = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model3, checkpoint_path, map_location='cpu')

model3.CLASSES = dataset.CLASSES
model3 = MMDataParallel(model3.cuda(), device_ids=[0])

2021-11-01 02:49:08,369 - mmseg - INFO - Loaded 819 images


Use load_from_local loader


In [5]:
output1 = single_gpu_test(model1, data_loader)
print("output1 done")
output2 = single_gpu_test(model2, data_loader)
print("output2 done")
output3 = single_gpu_test(model3, data_loader)
print("output3 done")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1515s, ETA:     0soutput1 done
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1516s, ETA:     0soutput2 done
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.5 task/s, elapsed: 1516s, ETA:     0soutput3 done


In [11]:
output = []

for predict1, predict2, predict3 in zip(output1, output2, output3):
  predict = predict1 + predict2 + predict3
  predict = predict.argmax(axis=0)
  output.append(predict)

output = np.array(output)
print(output.shape)

(819, 512, 512)


In [12]:
# sample_submisson.csv 열기
submission = pd.read_csv('../output/sample_submission.csv', index_col=None)
json_dir = os.path.join("../input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('../output/', f'ensemble_{epoch}.csv'), index=False)

In [ ]:
"""
# reference : https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
input_size = 512
output_size = 256
bin_size = input_size // output_size

for image_id, predict in enumerate(output):
  temp_mask = []
  predict = predict.reshape(1, 512, 512)
  print(predict.shape)
  mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2)
  print(mask.shape)
  temp_mask.append(mask)
  
  oms = np.array(temp_mask)
  oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
  print(oms.shape)
  predict = oms
  print(predict.shape)
"""